In [1]:
import math

import pandas
from fpgrowth_py import fpgrowth
import pandas as pd
import re
import json
import spacy
import seaborn as sns
from spacy import displacy
from flashtext import KeywordProcessor
import ahocorasick
import nltk
from concurrent.futures import ThreadPoolExecutor
from enum import Enum

pathprefix = r"D:\Code Working Area\Jupyter\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData"
jsfile = pathprefix+"\\Transactions.json"
stop_words=pathprefix+"\\stop_words\\stop_words_stakeholder.txt"
unknow_long_phrase=pathprefix+"\\stop_words\\unknow_long.txt"

project_sor = pathprefix+"\\Source\\project.csv"
risk_sor = pathprefix+"\\Source\\risk.xlsx"
stake_sor = pathprefix+"\\Source\\stakeholder.csv"

project_key = pathprefix+"\\project_keyword\\Project_keyword.xlsx"
risk_key_path = pathprefix+"\\risk_keyword\\Fourth_edition.xlsx"
stake_key = pathprefix+"\\stakeholder_keyword\\third_layer_iteration_one_stakeholder.xlsx"

In [2]:
prj_sor = pd.read_csv(project_sor, sep = ",")
risk_sor = pd.read_excel(risk_sor)
stk_sor = pd.read_csv(stake_sor, sep = ",")

prj_key = pd.read_excel(project_key, index_col=None)
risk_key = pd.read_excel(risk_key_path).dropna(axis=0, how="any")
stk_key = pd.read_excel(stake_key, index_col=None)
stk_key.dropna(inplace=True)

C:\Users\jqi22\AppData\Local\Temp\ipykernel_7032\2395300540.py:1: DtypeWarning: Columns (3,7,8,12,13,18,27,28,29,30,31,38,39,40,41,42,43,44,45,50,51,53,54,59,63,64,65,67) have mixed types. Specify dtype option on import or set low_memory=False.
  prj_sor = pd.read_csv(project_sor, sep = ",")


In [3]:
# get the target text from original dataset to match
project = pd.DataFrame(prj_sor["Article Title"])
risk = pd.DataFrame(risk_sor["Abstract"])
stake = pd.DataFrame(stk_sor["Abstract"])

risk.dropna(inplace=True, how="any")

In [4]:
stk_key.Abstract = stk_key.Abstract.str.lower()

# Do I really need below two steps?
risk_key.Abstract = risk_key.Abstract.astype(str)
stk_key.Abstract = stk_key.Abstract.astype(str)

In [5]:
# do this for keyword abstract
prjkey_list = prj_key["Article"].to_list()
riskey_list = risk_key["Abstract"].to_list()
stkey_list = stk_key.Abstract.to_list()
### initialize keywordprocessor class
keypro = KeywordProcessor()
keypro.add_keywords_from_list(prjkey_list + riskey_list + stkey_list)

In [6]:
nlp = spacy.load('en_core_web_sm') # disable=["attribute_ruler"]
stopwords = nlp.Defaults.stop_words
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [7]:
def reload_dashes(droped: int)->pd.DataFrame:
    dashes = pd.read_excel(pathprefix+"\\stop_words\\adjustment.xlsx")
    dashes = dashes[dashes.frequency > droped].Words.to_list()
    return set(dashes)

stopwords |= reload_dashes(2)
# filout = pd.read_excel(r"D:\Code Working Area\Python\knowledge-graph-for-stakeholder-risks-detection-in-mega-infrastructure-projects\ExcelData\filtered.xlsx")
# stopwords |= set(filout.name.to_list())
stopwords |= set([str(num) for num in range(1,100)])

# manually add the words to the stopwords
possiblew = {"connections", "efficacy", "life", "This"}
stopwords |= possiblew

for word in stopwords:
    nlp.vocab[word].is_stop=True

### extract keyword from Abstract

In [12]:
extracted=risk.Abstract.apply(lambda x: keypro.extract_keywords(x))

In [16]:
extracted=extracted.apply(lambda x: x if len(x)>0 else None).dropna()